# Extração dos dados das estações e S4
___

Os dados de estações e S4 estão organizados em um servidor postgresql rodando na minha maquina local. Para permitir a utilização por outros, este notebook, além de extrair os dados do postgresql vai gerar um csv, contendo as informações sobre as estações brutas. Assim, como um dataframes pandas, um por estação, comprimido usando o formatado xz, o que forneceu a extensão "pkl.xz".

As estações válidas são aquelas que apresentam algum dado de S4, pois existem estações que não coletaram dado nenhum durante o período de aquisição.

Note assim, que este notebook não executará em um máquina onde os dados não estejam armazenados no postgresql.

In [1]:
import aacgmv2
import os
import sys
import psycopg2

import datetime as dt
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from mpl_toolkits.basemap import Basemap
from itertools import chain
from tqdm import tqdm

from utils import local_data

%matplotlib inline

In [2]:
force = False

In [3]:
conn = psycopg2.connect(database="data-research",
                        user="data-research",
                        password="data-research",
                        host="localhost",
                        port=5432)

In [4]:
path = "./data/stations.csv"

if os.path.exists(path) and not force:
    df_station = pd.read_csv(path)
else:    
    df_station = pd.read_sql_query('select * from station;', con=conn)
    df_station.to_csv(path)

df_station = df_station.drop(columns=["startoperation", "stopoperation", "alias", "network", "Unnamed: 0"])    

In [5]:
query = '''select scintillation.eventdate, scintillation.s4 
           from scintillation inner join station on 
           scintillation.station = station.identificationstation 
           where station.identificationstation = '%s' 
           order by scintillation.eventdate; 
        '''
drop_aux = []
select_station_and_df_s4 = []
for i in tqdm(df_station.itertuples()):
    station = i.identificationstation
    path = "./data_s4/df_" + station + ".pkl.xz" 
    if not os.path.exists(path) or force:
        df_aux = pd.read_sql_query(query % station, con=conn)
        if len(df_aux) == 0:
            drop_aux.append(station)
        else:
            df_aux.to_pickle(path, compression='xz')

54it [00:00, 877.30it/s]


In [6]:
df_station = df_station[~df_station['identificationstation'].isin(drop_aux)]
df_station.to_pickle(os.path.join(local_data, 'df_station.pkl'))

In [7]:
df_station

,identificationstation,height,latitude,longitude,state,city,city_heigh,city_latitude,city_longitude
7,cpa,580.000,-22.410000,-45.000000,sp,cachoeira paulista,545.0,-22.67370,-44.9973
8,28,NaN,-3.100000,-59.976000,am,manaus,39.0,-3.10719,-60.0261
9,20,NaN,-20.497000,-54.615000,ms,campo grande,612.0,-20.44350,-54.6478
10,32,NaN,-20.497000,-54.615000,ms,campos grande,612.0,-20.44350,-54.6478
11,nta,0.000,-5.836162,-35.121000,rn,natal,38.0,-5.79448,-35.2110
12,sj2,593.440,-23.207600,-45.859800,sp,sao jose dos campos,593.0,-23.17910,-45.8872
14,16,NaN,-22.823000,-43.238000,rj,rio de janeiro (aeroporto galeão),20.0,-22.90350,-43.2096
15,22,NaN,-22.823000,-43.238000,rj,rio de janeiro (aeroporto galeão),20.0,-22.90350,-43.2096
16,cub,278.000,-15.555200,-56.069800,mt,cuiaba,180.0,-15.59890,-56.0949
17,dou,756.120,-22.110000,-54.550000,ms,dourados,448.0,-22.22180,-54.8064
